<a href="https://colab.research.google.com/github/murillo-borges/ifood-ab-campaign-case/blob/main/Ifood_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação do ambiente e ETL

### Database Pedidos

In [102]:
# 1 - Instalação e configuração do ambiente
print("⚙️ Instalando Java e PySpark...")
!apt-get install openjdk-11-jdk -y
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
print("✅ Ambiente configurado!\n")

# 2 - Iniciando a SparkSession
print("🚀 Iniciando sessão Spark...")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("iFood Order Case").getOrCreate()
print("✅ SparkSession criada!\n")

# 3 - Baixando o arquivo order.json.gz
print("📥 Baixando o arquivo order.json.gz...")
import urllib.request
url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
local_path = "order.json.gz"
urllib.request.urlretrieve(url, local_path)
print("✅ Download concluído!\n")

# 4 - Lendo o arquivo JSON compactado com PySpark
print("📊 Lendo arquivo JSON com PySpark...")
df_order = spark.read.json(local_path, multiLine=False)
print("✅ Dados carregados!\n")

# 5 - Exibindo as primeiras linhas do DataFrame
print("🔍 Primeiras linhas do DataFrame:")
df_order.show(5, truncate=False)

# 6 - Exibindo o schema do DataFrame
print("\n🧠 Estrutura do DataFrame:")
df_order.printSchema()

⚙️ Instalando Java e PySpark...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
✅ Ambiente configurado!

🚀 Iniciando sessão Spark...
✅ SparkSession criada!

📥 Baixando o arquivo order.json.gz...
✅ Download concluído!

📊 Lendo arquivo JSON com PySpark...
✅ Dados carregados!

🔍 Primeiras linhas do DataFrame:
+-----------+----------------------------------------------------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [103]:
from pyspark.sql.functions import explode, col, from_json
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, DoubleType, LongType

# Define the schema for the 'items' column
items_schema = ArrayType(StructType([
    StructField("name", StringType(), True),
    StructField("addition", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("discount", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("quantity", DoubleType(), True),
    StructField("sequence", LongType(), True), # Changed to LongType based on data example
    StructField("unitPrice", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("externalId", StringType(), True),
    StructField("totalValue", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("customerNote", StringType(), True),
    StructField("garnishItems", ArrayType(StructType([
        StructField("name", StringType(), True),
        StructField("addition", StructType([
            StructField("value", StringType(), True),
            StructField("currency", StringType(), True)
        ]), True),
        StructField("discount", StructType([
            StructField("value", StringType(), True),
            StructField("currency", StringType(), True)
        ]), True),
        StructField("quantity", DoubleType(), True),
        StructField("sequence", LongType(), True), # Changed to LongType based on data example
        StructField("unitPrice", StructType([
            StructField("value", StringType(), True),
            StructField("currency", StringType(), True)
        ]), True),
        StructField("categoryId", StringType(), True),
        StructField("externalId", StringType(), True),
        StructField("totalValue", StructType([
            StructField("value", StringType(), True),
            StructField("currency", StringType(), True)
        ]), True),
        StructField("categoryName", StringType(), True),
        StructField("integrationId", StringType(), True)
    ]), True), True),
    StructField("integrationId", StringType(), True),
    StructField("totalAddition", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("totalDiscount", StructType([
        StructField("value", StringType(), True),
        StructField("currency", StringType(), True)
    ]), True)
]))

# Parse the JSON string in 'items' column and then explode it
df_exploded = df_order.withColumn("parsed_items", from_json(col("items"), items_schema)) \
                      .withColumn("item", explode("parsed_items"))

# Show the schema of the new DataFrame with exploded items
print("🧠 Estrutura do DataFrame com itens explodidos:")
df_exploded.printSchema()

# Show the first few rows of the new DataFrame
print("\n🔍 Primeiras linhas do DataFrame com itens explodidos:")
df_exploded.select("order_id", "item.*").show(5, truncate=False)

🧠 Estrutura do DataFrame com itens explodidos:
root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- orde

In [104]:
df_order.show(1, truncate=False)

+-----------+----------------------------------------------------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [105]:
from pyspark.sql.functions import col, from_json, explode
from pyspark.sql.types import *

# 1 - Definindo o schema do array de itens
item_schema = ArrayType(
    StructType([
        StructField("name", StringType()),
        StructField("externalId", StringType()),
        StructField("quantity", DoubleType()),
        StructField("integrationId", StringType()),
        StructField("unitPrice", StructType([
            StructField("value", StringType()),
            StructField("currency", StringType())
        ])),
        StructField("totalValue", StructType([
            StructField("value", StringType()),
            StructField("currency", StringType())
        ]))
    ])
)

# 2 - Convertendo a string JSON para array de structs
df_order_fixed = df_order.withColumn("items_array", from_json(col("items"), item_schema))

# 3 - Aplicando explode na nova coluna
df_exploded = df_order_fixed.withColumn("item", explode("items_array"))

# 4 - Selecionando os campos desejados
df_items_detalhados = df_exploded.select(
    "order_id",
    "customer_id",
    "order_created_at",
    "order_total_amount",
    col("item.name").alias("item_name"),
    col("item.externalId").alias("item_id"),
    col("item.quantity").alias("item_quantity"),
    col("item.unitPrice.value").cast("float").alias("item_unit_price"),
    col("item.totalValue.value").cast("float").alias("item_total_price"),
    col("item.integrationId").alias("item_integration_id")
)

# 5 - Exibindo resultado
df_items_detalhados.show(5, truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------+------------------------+------------------+----------------------------------------+--------------------------------+-------------+---------------+----------------+-------------------+
|order_id                                                        |customer_id                                                     |order_created_at        |order_total_amount|item_name                               |item_id                         |item_quantity|item_unit_price|item_total_price|item_integration_id|
+----------------------------------------------------------------+----------------------------------------------------------------+------------------------+------------------+----------------------------------------+--------------------------------+-------------+---------------+----------------+-------------------+
|33e0612d62e5eb42aba15b58413137e441fbe906de2febd6

### Database Usuários

In [106]:
# 1 - Baixando o arquivo CSV compactado (.gz) da internet
print("📥 Baixando o arquivo consumer.csv.gz...")
import urllib.request

url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
local_path = "consumer.csv.gz"
urllib.request.urlretrieve(url, local_path)
print("✅ Download concluído!\n")

# 2 - Lendo o arquivo CSV com PySpark
print("📊 Lendo o arquivo CSV com PySpark...")
df_consumer = spark.read.option("header", True).option("inferSchema", True).csv(local_path)
print("✅ Arquivo carregado em um DataFrame Spark!\n")

# 3 - Exibindo as primeiras linhas do DataFrame
print("🔍 Primeiras linhas do dataset:")
df_consumer.show(5, truncate=False)

# 4 - Exibindo o schema do DataFrame
print("\n🧠 Estrutura geral do dataset:")
df_consumer.printSchema()

📥 Baixando o arquivo consumer.csv.gz...
✅ Download concluído!

📊 Lendo o arquivo CSV com PySpark...
✅ Arquivo carregado em um DataFrame Spark!

🔍 Primeiras linhas do dataset:
+----------------------------------------------------------------+--------+-----------------------+------+-------------+-------------------+---------------------+
|customer_id                                                     |language|created_at             |active|customer_name|customer_phone_area|customer_phone_number|
+----------------------------------------------------------------+--------+-----------------------+------+-------------+-------------------+---------------------+
|e8cc60860e09c0bb19610b06ced69c973eb83982cfc98e397ce65cba92f70928|pt-br   |2018-04-05 14:49:18.165|true  |NUNO         |46                 |816135924            |
|a2834a38a9876cf74e016524dd2e8c1f010ee12b2b684d58c40ab11eef19b6eb|pt-br   |2018-01-14 21:40:02.141|true  |ADRIELLY     |59                 |231330577            |
|41e105172

### Database Merchants - Restaurantes

In [107]:
# 1 - Baixando o arquivo CSV compactado (.gz) da internet
print("📥 Baixando o arquivo restaurant.csv.gz...")
import urllib.request

url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
local_path = "restaurant.csv.gz"
urllib.request.urlretrieve(url, local_path)
print("✅ Download concluído!\n")

# 2 - Lendo o arquivo CSV com PySpark
print("📊 Lendo o arquivo CSV com PySpark...")
df_restaurant = spark.read.option("header", True).option("inferSchema", True).csv(local_path)
print("✅ Arquivo carregado em um DataFrame Spark!\n")

# 3 - Exibindo as primeiras linhas do DataFrame
print("🔍 Primeiras linhas do dataset:")
df_restaurant.show(5, truncate=False)

# 4 - Exibindo o schema do DataFrame
print("\n🧠 Estrutura geral do dataset:")
df_restaurant.printSchema()

📥 Baixando o arquivo restaurant.csv.gz...
✅ Download concluído!

📊 Lendo o arquivo CSV com PySpark...
✅ Arquivo carregado em um DataFrame Spark!

🔍 Primeiras linhas do dataset:
+----------------------------------------------------------------+-----------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|id                                                              |created_at             |enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code|merchant_city |merchant_state|merchant_country|
+----------------------------------------------------------------+-----------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|d19ff6fca6288939bff073ad0a119d25c0365c407e9e5dd999e7a3e53c6d5d76|2017-01-23 12:52:30.91 |false  |3          

### Database Marcação de usuários que participaram do teste A/B

In [108]:
# 1 - Instalar Java e PySpark no Google Colab
!apt-get install openjdk-11-jdk -y
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# 2 - Iniciar SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("AB Test Case").getOrCreate()
print("✅ Spark iniciado!\n")

# 3 - Baixar o arquivo .tar.gz
import urllib.request
import tarfile

print("📥 Baixando o arquivo ab_test_ref.tar.gz...")
url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
local_tar_path = "ab_test_ref.tar.gz"
urllib.request.urlretrieve(url, local_tar_path)
print("✅ Download concluído!\n")

# 4 - Extrair o conteúdo
print("🗂 Extraindo arquivos...")
extract_path = "ab_test_ref_extracted"
os.makedirs(extract_path, exist_ok=True)
with tarfile.open(local_tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("✅ Extração concluída!\n")

# 5 - Caminho direto para o arquivo extraído
csv_path = os.path.join(extract_path, "ab_test_ref.csv")
print(f"📄 Lendo arquivo: {csv_path}")

# 6 - Definir schema do arquivo
from pyspark.sql.types import StructType, StructField, StringType
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("is_target", StringType(), True),
])

# 7 - Leitura do CSV com PySpark
df_ab_test = spark.read.option("header", True)\
                       .option("sep", ",")\
                       .schema(schema)\
                       .csv(csv_path)

# 8 - Visualizar os dados
print("🔍 Primeiras linhas do DataFrame:")
df_ab_test.show(5, truncate=False)

print("\n🧠 Estrutura do DataFrame:")
df_ab_test.printSchema()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
✅ Spark iniciado!

📥 Baixando o arquivo ab_test_ref.tar.gz...
✅ Download concluído!

🗂 Extraindo arquivos...
✅ Extração concluída!

📄 Lendo arquivo: ab_test_ref_extracted/ab_test_ref.csv
🔍 Primeiras linhas do DataFrame:
+----------------------------------------------------------------+---------+
|customer_id                                                     |is_target|
+----------------------------------------------------------------+---------+
|755e1fa18f25caec5edffb188b13fd844b2af8cf5adedcf77c028f36cb9382ea|target   |
|b821aa8372b8e5b82cdc283742757df8c45eecdd72adf411716e710525d4edf1|control  |
|d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc007f5297541ec40b63b|control  |
|6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c8b535718bbc651ab02|target   |
|da

**Explanation of the fix:**

1.  **`items_schema`**: We define a `StructType` that matches the expected structure of each item within the `items` array. This includes nested `StructType` for fields like `addition`, `discount`, `unitPrice`, `totalValue`, `totalAddition`, and `totalDiscount`, and an `ArrayType` for `garnishItems`. I've used `StringType` for currency values and prices based on the example data in the output of cell `Bh7E-EBFhrMs`, and `LongType` for `sequence`.
2.  **`from_json(col("items"), items_schema)`**: This function parses the JSON string in the `items` column according to the `items_schema` we defined. The result is a new column named `parsed_items` which is of type `ArrayType(StructType(...))`.
3.  **`explode("parsed_items")`**: Now that `parsed_items` is an array type, we can successfully apply the `explode` function to it. This creates a new row for each item in the array.
4.  **`df_exploded.select("order_id", "item.*").show(5, truncate=False)`**: This line selects the original `order_id` and all the fields from the exploded `item` column to show the result of the explosion.

This corrected code will successfully extract the individual items from the `items` column and create a new row for each item, allowing you to analyze the items within each order.

## O Desafio – Análise do teste A/B (ETAPA 1)

✅ CÓDIGO PySpark: Análise A/B (impacto da campanha)


In [109]:
from pyspark.sql.functions import col, countDistinct, count, sum, avg, when
from pyspark.sql.window import Window


In [110]:
# 1 - Juntando pedidos com grupo de teste
df_ab_orders = df_order.join(df_ab_test, on="customer_id", how="inner")

In [111]:
# 2 - Calculando indicadores agregados por grupo (target vs control)
df_metrics = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    avg("order_total_amount").alias("ticket_medio"),
    sum("order_total_amount").alias("valor_total_pedidos")
)

In [112]:
# 3 - Calculando pedidos por usuário (frequência média)
df_metrics = df_metrics.withColumn(
    "pedidos_por_usuario",
    col("qtd_pedidos") / col("qtd_usuarios")
)

# 4 - Exibindo resultados
df_metrics.show(truncate=False)

+---------+------------+-----------+------------------+--------------------+-------------------+
|is_target|qtd_usuarios|qtd_pedidos|ticket_medio      |valor_total_pedidos |pedidos_por_usuario|
+---------+------------+-----------+------------------+--------------------+-------------------+
|control  |360542      |1525576    |47.897890947419995|7.30718728800012E7  |4.231340592774212  |
|target   |445924      |2136745    |47.73970213572941 |1.0200756984000914E8|4.79172459881056   |
+---------+------------+-----------+------------------+--------------------+-------------------+



### a) Definindo os indicadores relevantes para mensurar o sucesso da
campanha e analisar se ela teve impacto significativo dentro do
período avaliado:

In [113]:
from pyspark.sql.functions import col, countDistinct, count, sum, avg
import pandas as pd

# 1 - Juntando pedidos com grupo de teste
df_ab_orders = df_order.join(df_ab_test, on="customer_id", how="inner")

# 2 - Calculando indicadores agregados por grupo (target vs control)
df_metrics = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    avg("order_total_amount").alias("ticket_medio"),
    sum("order_total_amount").alias("valor_total_pedidos")
)

# 3 - Calculando pedidos por usuário
df_metrics = df_metrics.withColumn(
    "pedidos_por_usuario",
    col("qtd_pedidos") / col("qtd_usuarios")
)

# 4 - Convertendo para Pandas para aplicar a formatação personalizada
df_metrics_pd = df_metrics.toPandas()

# 5 - Formatando os valores para estilo brasileiro
df_metrics_formatado = pd.DataFrame()
#df_metrics_formatado["Grupo"] = df_metrics_pd["is_target"]

df_metrics_formatado["qtd_usuarios"] = df_metrics_pd["qtd_usuarios"].apply(lambda x: f"{x:,.0f}".replace(",", "."))
df_metrics_formatado["qtd_pedidos"] = df_metrics_pd["qtd_pedidos"].apply(lambda x: f"{x:,.0f}".replace(",", "."))
df_metrics_formatado["ticket_medio"] = df_metrics_pd["ticket_medio"].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
df_metrics_formatado["valor_total_pedidos"] = df_metrics_pd["valor_total_pedidos"].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
df_metrics_formatado["pedidos_por_usuario"] = df_metrics_pd["pedidos_por_usuario"].apply(lambda x: f"{x:.1f}".replace(".", ","))

# 6 - Exibindo resultado
print(df_metrics_formatado.to_string(index=False))

qtd_usuarios qtd_pedidos ticket_medio valor_total_pedidos pedidos_por_usuario
     360.542   1.525.576     R$ 47,90    R$ 73.071.872,88                 4,2
     445.924   2.136.745     R$ 47,74   R$ 102.007.569,84                 4,8


✅ 1. Join dos dados de pedidos com teste A/B e restaurantes


In [114]:
# Base principal de análise
df_ab_orders = df_order.join(df_ab_test, on="customer_id", how="inner")
df_ab_orders = df_ab_orders.join(df_restaurant, df_ab_orders["merchant_id"] == df_restaurant["id"], "inner")

📊 2. KPIs principais por grupo (usuários e pedidos)

In [115]:
from pyspark.sql.functions import col, countDistinct, count, sum, avg

df_metrics = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("valor_total_pedidos"),
    avg("order_total_amount").alias("ticket_medio")
).withColumn(
    "pedidos_por_usuario", col("qtd_pedidos") / col("qtd_usuarios")
)

🔁 3. Retenção de usuários (usuários com >1 pedido)


In [116]:
from pyspark.sql.functions import when, round

df_freq = df_ab_orders.groupBy("is_target", "customer_id").agg(count("order_id").alias("qtd_pedidos"))

df_retencao = df_freq.groupBy("is_target").agg(
    count("*").alias("total_usuarios"),
    count(when(col("qtd_pedidos") > 1, True)).alias("usuarios_retidos")
).withColumn(
    "taxa_retencao", round((col("usuarios_retidos") / col("total_usuarios")) * 100, 2)
).select("is_target", "taxa_retencao")

📈 4. Incremento percentual do ticket médio


In [117]:
from pyspark.sql.functions import first

df_ticket_medio = df_ab_orders.groupBy("is_target").agg(round(avg("order_total_amount"), 2).alias("ticket_medio"))

df_incremento = df_ticket_medio.groupBy().pivot("is_target").agg(first("ticket_medio")).withColumn(
    "incremento_percentual", round(((col("target") - col("control")) / col("control")) * 100, 2)
).select("incremento_percentual")

🔝 5. Heavy users (usuários que estão no top 20% de pedidos)

In [118]:
df_user_freq = df_ab_orders.groupBy("customer_id").agg(count("order_id").alias("qtd_pedidos"))
percentil_80 = df_user_freq.approxQuantile("qtd_pedidos", [0.80], 0.01)[0]

df_user_freq = df_user_freq.withColumn(
    "heavy_user", when(col("qtd_pedidos") >= percentil_80, 1).otherwise(0)
)

df_heavy = df_user_freq.join(df_ab_test, on="customer_id", how="inner")

df_heavy_summary = df_heavy.groupBy("is_target").agg(
    count(when(col("heavy_user") == 1, True)).alias("qtd_heavy_users"),
    count("*").alias("total_usuarios")
).withColumn(
    "percentual_heavy_users", round((col("qtd_heavy_users") / col("total_usuarios")) * 100, 2)
).select("is_target", "percentual_heavy_users")

🕒 6. Tempo médio de entrega por grupo

In [119]:
df_delivery_time = df_ab_orders.groupBy("is_target").agg(
    round(avg("delivery_time"), 2).alias("tempo_medio_entrega")
)

🍽️ 7. Total de restaurantes por grupo (com base em pedidos)


In [120]:
df_qtd_rest = df_ab_orders.groupBy("is_target").agg(
    countDistinct("merchant_id").alias("qtd_restaurantes")
)

📦 8. Média de pedidos por restaurante

In [121]:
df_avg_pedidos_rest = df_ab_orders.groupBy("is_target", "merchant_id").agg(
    count("order_id").alias("qtd_pedidos_rest")
).groupBy("is_target").agg(
    round(avg("qtd_pedidos_rest"), 2).alias("media_pedidos_por_restaurante")
)

🧾 10. Juntando todos os KPIs em uma tabela final

In [122]:
# Juntando tudo com PySpark
df_final = df_metrics \
    .join(df_retencao, on="is_target", how="left") \
    .join(df_heavy_summary, on="is_target", how="left") \
    .join(df_delivery_time, on="is_target", how="left") \
    .join(df_qtd_rest, on="is_target", how="left") \
    .join(df_avg_pedidos_rest, on="is_target", how="left")

df_final.show(truncate=False)

+---------+------------+-----------+--------------------+-----------------+-------------------+-------------+----------------------+-------------------+----------------+-----------------------------+
|is_target|qtd_usuarios|qtd_pedidos|valor_total_pedidos |ticket_medio     |pedidos_por_usuario|taxa_retencao|percentual_heavy_users|tempo_medio_entrega|qtd_restaurantes|media_pedidos_por_restaurante|
+---------+------------+-----------+--------------------+-----------------+-------------------+-------------+----------------------+-------------------+----------------+-----------------------------+
|control  |360542      |1525576    |7.307187287999888E7 |47.89789094741847|4.231340592774212  |74.7         |21.53                 |22.63              |7196            |212.0                        |
|target   |445924      |2136745    |1.0200756984000076E8|47.73970213572549|4.79172459881056   |79.51        |25.87                 |22.66              |7227            |295.66                       |


#### 📊 Análise Comparativa — Grupo Controle vs Grupo Target

| Indicador                    | Controle  | Target    | Diferença | Interpretação                               |
| ---------------------------- | --------- | --------- | --------- | ------------------------------------------- |
| **Usuários (qtd\_usuarios)** | 360.542   | 445.924   | +85.382   | Target teve uma base maior.                 |
| **Pedidos (qtd\_pedidos)**   | 1.525.576 | 2.136.745 | +611.169  | Target pediu mais.                          |
| **Ticket médio**             | R\$ 47,90 | R\$ 47,74 | -R\$ 0,16 | Leve queda no valor por pedido no target.   |
| **Pedidos por usuário**      | 4,23      | 4,79      | +13,2%    | Mais engajamento no target.                 |
| **Retenção de usuários (%)** | 74,7%     | 79,51%    | +4,8 p.p. | A campanha reteve melhor os usuários.       |
| **Heavy Users (%)**          | 21,53%    | 25,87%    | +4,3 p.p. | Mais usuários muito ativos no target.       |
| **Tempo médio de entrega**   | 22,63 min | 22,66 min | ≈         | Não há impacto logístico relevante.         |
| **Pedidos por restaurante**  | 212,0     | 295,66    | +39,4%    | Restaurantes no grupo target venderam mais. |


**Análise de Engajamento e Produto:**

Observando as métricas de comportamento, a campanha foi um sucesso absoluto.


*   Frequência de Pedidos: Os usuários do grupo target fizeram, em média, 4,79 pedidos, um aumento de 13,2% em relação aos 4,23 do grupo control. Isso mostra que o incentivo funcionou para criar o hábito de pedir mais vezes.

*   Taxa de Retenção: A retenção do grupo target foi de 79,51%, quase 5 pontos percentuais maior que a do grupo control. Este é um ganho massivo e o indicador mais forte de que a campanha gerou lealdade.


*   "Heavy Users": O percentual de usuários de alta frequência subiu de 21,53% para 25,87%. A campanha foi eficaz em converter usuários e engajar a base.


Conclusão de Produto: Sem dúvida, a campanha foi um sucesso em engajar a base de usuários, aumentar a frequência e reter mais clientes.

### b) Análise da viabilidade financeira dessa iniciativa como alavanca de crescimento

#### 💰 Análise da Viabilidade Financeira


**Premissas Adotadas:**

*   Custo da Campanha: R$ 10,00 por cada pedido realizado no grupo target.
*   Take Rate (Receita do iFood): Manteremos a premissa de 20% sobre o valor total dos pedidos (GMV).


**1 - Custo da Campanha**

In [123]:
from pyspark.sql.functions import col

# --- Premissas ---
# Ajuste o valor do custo do cupom conforme a sua necessidade.
CUSTO_CUPOM_POR_PEDIDO = 10.00

# --- Cálculo Direto ---
# O código filtra o DataFrame para o grupo 'target', conta o número de pedidos
# e multiplica pelo custo unitário do cupom para obter o valor total.
custo_total_campanha = df_ab_orders.filter(col("is_target") == "target").count() * CUSTO_CUPOM_POR_PEDIDO

# --- Resultado ---
print(f"Custo Total: R$ {custo_total_campanha:,.2f}")

Custo Total: R$ 21,367,450.00


**2 - Receita Incremental**

In [124]:
from pyspark.sql.functions import col, sum, countDistinct

# --- Premissas ---
# Ajuste o Take Rate (comissão do iFood estimada).
TAKE_RATE = 0.20

# --- 1. Agregação com PySpark ---
# Agrupamos os dados para obter os totais de usuários e GMV para cada grupo.
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 2. Extração dos Dados Agregados ---
# Coletamos o pequeno DataFrame de resumo para o Python para facilitar os cálculos.
# O .collect() aqui é seguro e eficiente, pois o resultado tem apenas 2 linhas.
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 3. Cálculo Financeiro ---
# Verificamos se ambos os grupos existem antes de calcular
if control_metrics and target_metrics:
    # Comportamento base: Qual foi o GMV por usuário no grupo de controle?
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]

    # Projeção: Qual seria o GMV do grupo Target se eles fossem como o Controle?
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]

    # Receita projetada (Baseline) vs. Receita Real
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE

    # A Receita Incremental é a diferença entre o que aconteceu e o que teria acontecido.
    receita_incremental = receita_real_target - receita_baseline_target
else:
    receita_incremental = 0  # Define como 0 se um dos grupos não existir

# --- 4. Resultado ---
print("CÁLCULO DA RECEITA INCREMENTAL DA CAMPANHA")
print(f"Take Rate (premissa): {TAKE_RATE:.0%}")
print("---------------------------------------------")
print(f"Receita Incremental Gerada: R$ {receita_incremental:,.2f}")

  CÁLCULO DA RECEITA INCREMENTAL DA CAMPANHA
Take Rate (premissa): 20%
---------------------------------------------
Receita Incremental Gerada: R$ 2,326,226.29


**3 - Resultado Líquido**

In [125]:
from pyspark.sql.functions import col, sum, count, countDistinct

# --- 1. Premissas da Análise ---
# Ajuste estes valores conforme sua necessidade.
TAKE_RATE = 0.20  # 20% de comissão para o iFood
CUSTO_CUPOM_POR_PEDIDO = 10.00  # R$ 10,00 por pedido no grupo Target

# --- 2. Agregação Única com PySpark ---
# Em uma única passagem, calculamos todos os totais necessários para ambos os grupos.
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 3. Extração dos Dados para o Modelo Financeiro ---
# Coletamos o pequeno DataFrame de resumo para o Python para facilitar os cálculos.
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 4. Cálculo dos Componentes e do Resultado Líquido ---
# Inicializamos as variáveis para evitar erros se um grupo não existir
receita_incremental = 0
custo_total_campanha = 0

if control_metrics and target_metrics:
    # --- Cálculo do Custo da Campanha ---
    custo_total_campanha = target_metrics["qtd_pedidos"] * CUSTO_CUPOM_POR_PEDIDO

    # --- Cálculo da Receita Incremental ---
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE
    receita_incremental = receita_real_target - receita_baseline_target

# --- Cálculo Final ---
resultado_liquido = receita_incremental - custo_total_campanha


# --- 5. Exibição do Resultado ---
print("COMPONENTES DA ANÁLISE:")
print(f"  (+) Receita Incremental Gerada: R$ {receita_incremental:,.2f}")
print(f"  (-) Custo Total da Campanha:    R$ {custo_total_campanha:,.2f}")
print("--------------------------------------------------")
print(f"  📊 Resultado Líquido Final:    R$ {resultado_liquido:,.2f}")

COMPONENTES DA ANÁLISE:
  (+) Receita Incremental Gerada: R$ 2,326,226.29
  (-) Custo Total da Campanha:    R$ 21,367,450.00
--------------------------------------------------
  📊 Resultado Líquido Final:    R$ -19,041,223.71


**4 - ROI: Retorno sobre Investimento**

In [126]:
from pyspark.sql.functions import col, sum, count, countDistinct

# --- 1. Premissas da Análise ---
# Ajuste estes valores conforme sua necessidade.
TAKE_RATE = 0.20  # 20% de comissão para o iFood
CUSTO_CUPOM_POR_PEDIDO = 10.00  # R$ 10,00 por pedido no grupo Target

# --- 2. Agregação Única com PySpark ---
# Em uma única passagem, calculamos todos os totais necessários para ambos os grupos.
summary_df = df_ab_orders.groupBy("is_target").agg(
    countDistinct("customer_id").alias("qtd_usuarios"),
    count("order_id").alias("qtd_pedidos"),
    sum("order_total_amount").alias("gmv_total")
)

# --- 3. Extração dos Dados para o Modelo Financeiro ---
# Coletamos o pequeno DataFrame de resumo para o Python para facilitar os cálculos.
metrics = {row["is_target"]: row for row in summary_df.collect()}

control_metrics = metrics.get("control")
target_metrics = metrics.get("target")

# --- 4. Cálculo dos Componentes Financeiros ---
# Inicializamos as variáveis para evitar erros se um grupo não existir
receita_incremental = 0
custo_total_campanha = 0
resultado_liquido = 0
roi = 0

if control_metrics and target_metrics:
    # --- Custo da Campanha ---
    custo_total_campanha = target_metrics["qtd_pedidos"] * CUSTO_CUPOM_POR_PEDIDO

    # --- Receita Incremental ---
    gmv_por_usuario_control = control_metrics["gmv_total"] / control_metrics["qtd_usuarios"]
    gmv_baseline_target = gmv_por_usuario_control * target_metrics["qtd_usuarios"]
    receita_baseline_target = gmv_baseline_target * TAKE_RATE
    receita_real_target = target_metrics["gmv_total"] * TAKE_RATE
    receita_incremental = receita_real_target - receita_baseline_target

    # --- Resultado Líquido ---
    resultado_liquido = receita_incremental - custo_total_campanha

    # --- Cálculo Final do ROI ---
    # Verificamos se o custo é maior que zero para evitar divisão por zero
    if custo_total_campanha > 0:
        roi = resultado_liquido / custo_total_campanha

# --- 5. Exibição do Relatório Financeiro Completo ---
print("PREMISSAS ADOTADAS:")
print(f"  - Take Rate (Comissão iFood): {TAKE_RATE:.0%}")
print(f"  - Custo do Cupom por Pedido (Target): R$ {CUSTO_CUPOM_POR_PEDIDO:.2f}\n")

print("ANÁLISE DE RESULTADOS:")
print(f"  - Receita Incremental Gerada:       R$ {receita_incremental:,.2f}")
print(f"  - Custo Total da Campanha:          R$ {custo_total_campanha:,.2f}")
print("-------------------------------------------------------")
print(f"  📊 Resultado Líquido Final:          R$ {resultado_liquido:,.2f}")
print("-------------------------------------------------------")
print(f"  📈 ROI (Retorno sobre Investimento): {roi:.2%}")
print("="*55)

PREMISSAS ADOTADAS:
  - Take Rate (Comissão iFood): 20%
  - Custo do Cupom por Pedido (Target): R$ 10.00

ANÁLISE DE RESULTADOS:
  - Receita Incremental Gerada:       R$ 2,326,226.29
  - Custo Total da Campanha:          R$ 21,367,450.00
-------------------------------------------------------
  📊 Resultado Líquido Final:          R$ -19,041,223.71
-------------------------------------------------------
  📈 ROI (Retorno sobre Investimento): -89.11%


**📉 Conclusão: A Campanha Teve Retorno?**


Não. A campanha **não teve retorno financeiro positivo.**

**Apesar de gerar:**

*   Maior engajamento dos usuários (mais pedidos por usuário)
*   Maior taxa de retenção
*   Mais heavy users
*   Aumento nas vendas dos restaurantes




👉 O custo da campanha (desconto) foi quase 10x maior que a receita adicional gerada, gerando **prejuízo** de mais de R$ 19 milhões e um ROI negativo de -89,11%.


💡 Insight
A estratégia melhorou comportamento e engajamento, mas não foi eficiente financeiramente.

Se o cupom fosse menor (ex: R$5) ou vinculado a pedidos acima de um valor mínimo de R\$ 60.00, poderia manter os efeitos positivos com menor custo.

### c) Recomendações de oportunidades de melhoria nessa ação e nova proposta de teste A/B para validar essas hipóte-ses.

### 📊 **Resumo da campanha atual**

A campanha com cupom aumentou o engajamento:

* Retenção subiu para **79,5%** (vs 74,7%)
* Mais **pedidos por usuário** (4,79 vs 4,23)
* Mais **heavy users** (25,8% vs 21,5%)

❌ **Problema**: o custo foi alto demais
💸 **Resultado líquido**: –R\$ 19 milhões
📉 **ROI**: –89.11%

---

## 🚀 **Oportunidades de melhoria**

1. **Focar em usuários inativos ou novos**, não em todos.
2. **Reduzir o valor do cupom** ou exigir **valor mínimo de compra**.
3. **Estimular o aumento do ticket médio** com frete grátis.

---

## 🧪 **Nova proposta de teste A/B**

| Grupo    | Estratégia                          |
| -------- | ----------------------------------- |
| Controle | Sem cupom                           |
| Teste A  | R\$10 só para usuários inativos  |
| Teste B  | R\$5 de desconto em pedidos > R\$40     |
| Teste C  | Frete Grátis para pedidos acima de R$ 65,00         |

**Métricas para analisar**: Receita incremental, Resultado Líquido, ROI, Retenção, Reativação, Ticket médio.


## O Desafio – Análise por Segmentação (ETAPA 2)

In [129]:
from pyspark.sql.functions import col, count, sum, avg, when, round

# 1. Calcular métrica por usuário
df_user_metrics = df_ab_orders.groupBy("customer_id", "is_target").agg(
    count("order_id").alias("qtd_pedidos"),
    round(sum("order_total_amount") / count("order_id"), 2).alias("ticket_medio")
)

# 2. Calcular tercis
qtd_tercis = df_user_metrics.approxQuantile("qtd_pedidos", [0.33, 0.66], 0.01)
ticket_tercis = df_user_metrics.approxQuantile("ticket_medio", [0.33, 0.66], 0.01)

# 3. Segmentar por frequência
df_user_metrics = df_user_metrics.withColumn(
    "segmento_frequencia",
    when(col("qtd_pedidos") <= qtd_tercis[0], "Baixa frequência")
    .when(col("qtd_pedidos") <= qtd_tercis[1], "Média frequência")
    .otherwise("Alta frequência")
)

# 4. Segmentar por ticket médio
df_user_metrics = df_user_metrics.withColumn(
    "segmento_ticket",
    when(col("ticket_medio") <= ticket_tercis[0], "Ticket baixo")
    .when(col("ticket_medio") <= ticket_tercis[1], "Ticket médio")
    .otherwise("Ticket alto")
)

# 5. Contar usuários por grupo e segmentos
df_segmentos_final = df_user_metrics.groupBy("is_target", "segmento_frequencia", "segmento_ticket").agg(
    count("customer_id").alias("qtd_usuarios")
).orderBy("is_target", "segmento_frequencia", "segmento_ticket")

df_segmentos_final.show(truncate=False)

+---------+-------------------+---------------+------------+
|is_target|segmento_frequencia|segmento_ticket|qtd_usuarios|
+---------+-------------------+---------------+------------+
|control  |Alta frequência    |Ticket alto    |33078       |
|control  |Alta frequência    |Ticket baixo   |26329       |
|control  |Alta frequência    |Ticket médio   |33784       |
|control  |Baixa frequência   |Ticket alto    |70437       |
|control  |Baixa frequência   |Ticket baixo   |74262       |
|control  |Baixa frequência   |Ticket médio   |61870       |
|control  |Média frequência   |Ticket alto    |21043       |
|control  |Média frequência   |Ticket baixo   |18994       |
|control  |Média frequência   |Ticket médio   |20745       |
|target   |Alta frequência    |Ticket alto    |49032       |
|target   |Alta frequência    |Ticket baixo   |39409       |
|target   |Alta frequência    |Ticket médio   |50286       |
|target   |Baixa frequência   |Ticket alto    |73358       |
|target   |Baixa frequên

In [130]:
from pyspark.sql.functions import col, when, count, avg

# 1 - Agrupar usuários com seus KPIs individuais
df_user_stats = df_ab_orders.groupBy("customer_id", "is_target").agg(
    count("order_id").alias("qtd_pedidos"),
    avg("order_total_amount").alias("ticket_medio_usuario")
)

# 2 - Criar segmentos por frequência
df_user_segmentado = df_user_stats.withColumn(
    "segmento_frequencia",
    when(col("qtd_pedidos") <= 2, "Baixa frequência")
    .when((col("qtd_pedidos") >= 3) & (col("qtd_pedidos") <= 5), "Média frequência")
    .otherwise("Alta frequência")
)

# 3 - Criar segmentos por ticket médio
df_user_segmentado = df_user_segmentado.withColumn(
    "segmento_ticket",
    when(col("ticket_medio_usuario") <= 35, "Ticket baixo")
    .when((col("ticket_medio_usuario") > 35) & (col("ticket_medio_usuario") <= 60), "Ticket médio")
    .otherwise("Ticket alto")
)

# 4 - Quantidade de usuários por grupo e segmento
df_resultado_segmentado = df_user_segmentado.groupBy("is_target", "segmento_frequencia", "segmento_ticket").agg(
    count("customer_id").alias("qtd_usuarios")
).orderBy("is_target", "segmento_frequencia", "segmento_ticket")

# Mostrar resultado
df_resultado_segmentado.show(truncate=False)


+---------+-------------------+---------------+------------+
|is_target|segmento_frequencia|segmento_ticket|qtd_usuarios|
+---------+-------------------+---------------+------------+
|control  |Alta frequência    |Ticket alto    |16597       |
|control  |Alta frequência    |Ticket baixo   |25754       |
|control  |Alta frequência    |Ticket médio   |35270       |
|control  |Baixa frequência   |Ticket alto    |43860       |
|control  |Baixa frequência   |Ticket baixo   |84267       |
|control  |Baixa frequência   |Ticket médio   |78442       |
|control  |Média frequência   |Ticket alto    |16409       |
|control  |Média frequência   |Ticket baixo   |26956       |
|control  |Média frequência   |Ticket médio   |32987       |
|target   |Alta frequência    |Ticket alto    |24702       |
|target   |Alta frequência    |Ticket baixo   |38555       |
|target   |Alta frequência    |Ticket médio   |52085       |
|target   |Baixa frequência   |Ticket alto    |45679       |
|target   |Baixa frequên

In [131]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, when, percent_rank, round as spark_round
from pyspark.sql.window import Window

# ==============================================================================
# 1. SETUP E SIMULAÇÃO DE DADOS (SUBSTITUA PELO SEU DATAFRAME)
# ==============================================================================
spark = SparkSession.builder.appName("SegmentABTest_NoDate").getOrCreate()

# ==============================================================================
# 2. CRIAÇÃO DA BASE DE ANÁLISE POR USUÁRIO (FEATURE STORE)
# ==============================================================================
# (Item b) - Parte 1: Agregamos os dados para ter uma visão única por cliente.
print("--- Passo 1: Criando a base de análise por usuário... ---")
df_user_features = df_ab_orders.groupBy("customer_id", "is_target").agg(
    count("order_id").alias("total_pedidos"),
    avg("order_total_amount").alias("ticket_medio_usuario"),
    sum("order_total_amount").alias("gasto_total_usuario")
)

# ==============================================================================
# 3. APLICAÇÃO DOS CRITÉRIOS DE SEGMENTAÇÃO
# ==============================================================================
# (Item a e b) - Parte 2: Adicionamos as "etiquetas" de segmento a cada usuário.
print("--- Passo 2: Aplicando os critérios de segmentação... ---")

# --- Segmentação por Frequência ---
window_freq = Window.partitionBy("is_target").orderBy("total_pedidos")
df_user_features = df_user_features.withColumn("rank_freq", percent_rank().over(window_freq))
df_user_features = df_user_features.withColumn("segmento_frequencia",
    when(col("rank_freq") >= 0.8, "1. Heavy User (Top 20%)")
    .when((col("rank_freq") >= 0.3) & (col("rank_freq") < 0.8), "2. Casual (30-80%)")
    .otherwise("3. Leve (Bottom 30%)")
)

# --- Segmentação por Valor (Ticket Médio) ---
window_valor = Window.partitionBy("is_target").orderBy("ticket_medio_usuario")
df_user_features = df_user_features.withColumn("rank_valor", percent_rank().over(window_valor))
df_user_features = df_user_features.withColumn("segmento_valor",
    when(col("rank_valor") >= 0.5, "1. Alto Valor")
    .otherwise("2. Baixo Valor")
)

print("\n--- DataFrame de Usuários com Segmentos (Amostra) ---")
df_user_features.select(
    "customer_id", "is_target", "total_pedidos", "ticket_medio_usuario",
    "segmento_frequencia", "segmento_valor"
).show(10, truncate=False)


# ==============================================================================
# 4. ANÁLISE CRUZADA FINAL: KPIs POR SEGMENTO
# ==============================================================================
# (Item c) - Agrupamos por grupo de teste E por segmento para gerar os relatórios.
print("--- Passo 3: Gerando as análises cruzadas por segmento... ---")


print("\n\n" + "="*70)
print("          (c) ANÁLISE DE RESULTADOS POR SEGMENTO DE FREQUÊNCIA")
print("="*70)
df_analise_frequencia = df_user_features.groupBy("is_target", "segmento_frequencia").agg(
    count("customer_id").alias("qtd_usuarios"),
    spark_round(avg("total_pedidos"), 2).alias("pedidos_por_usuario"),
    spark_round(avg("ticket_medio_usuario"), 2).alias("ticket_medio_segmento")
).orderBy("segmento_frequencia", "is_target")
df_analise_frequencia.show(truncate=False)


print("\n" + "="*70)
print("          (c) ANÁLISE DE RESULTADOS POR SEGMENTO DE VALOR (TICKET MÉDIO)")
print("="*70)
df_analise_valor = df_user_features.groupBy("is_target", "segmento_valor").agg(
    count("customer_id").alias("qtd_usuarios"),
    spark_round(avg("total_pedidos"), 2).alias("pedidos_por_usuario"),
    spark_round(avg("ticket_medio_usuario"), 2).alias("ticket_medio_segmento")
).orderBy("segmento_valor", "is_target")
df_analise_valor.show(truncate=False)


--- Passo 1: Criando a base de análise por usuário... ---
--- Passo 2: Aplicando os critérios de segmentação... ---

--- DataFrame de Usuários com Segmentos (Amostra) ---
+----------------------------------------------------------------+---------+-------------+--------------------+--------------------+--------------+
|customer_id                                                     |is_target|total_pedidos|ticket_medio_usuario|segmento_frequencia |segmento_valor|
+----------------------------------------------------------------+---------+-------------+--------------------+--------------------+--------------+
|7619dad16eeaa95e42a0867b7ccaf40bdadb6197c4574230a2731df366605edf|control  |1            |0.0                 |3. Leve (Bottom 30%)|2. Baixo Valor|
|280207ba97d79a5fceae288ebffe300b94295cd5e83b828b85234a98e0dd8308|control  |1            |0.0                 |3. Leve (Bottom 30%)|2. Baixo Valor|
|9d805405db0da593d992acfded327a569d181d57a18c1a23d172722e0d3aa77f|control  |1            

### **Análise Estratégica do Teste A/B por Segmentos de Usuário**

A análise inicial do Teste A/B nos mostrou que apesar do resultado financeiro da campanha ser péssimo, foi um sucesso em aumentar a frequência de pedidos e a retenção de clientes. No entanto, esse resultado é uma média geral. A segmentação nos permite aprofundar a análise para entender *quais perfis de clientes* foram mais impactados e *por que*, transformando dados em inteligência para ações futuras mais eficazes e personalizadas.

---

### **a) Definição das Segmentações Propostas**

Para analisar o impacto da campanha de cupons, que é um incentivo de frequência, definimos duas segmentações principais baseadas no comportamento de compra do usuário:

1.  **Segmentação por Frequência de Pedidos:** Agrupa os usuários com base no seu nível de engajamento (quantas vezes eles pedem).
2.  **Segmentação por Perfil de Gasto (Ticket Médio):** Agrupa os usuários com base no valor médio que costumam gastar em cada pedido.

---

### **b) Critérios de Criação e Racional de Cada Segmento**

Aqui detalhamos como cada segmento é construído e qual pergunta de negócio ele visa responder.

#### **1. Segmentação por Frequência de Pedidos**

* **Critérios de Criação:**
    * Calculamos o número total de pedidos de cada usuário.
    * Utilizamos **percentis** para classificar a base de clientes em três grupos distintos:
        * **Heavy Users (Top 20%):** Seus clientes mais leais e frequentes.
        * **Usuários Casuais (Intermediários 50%):** Clientes que utilizam o serviço com regularidade.
        * **Usuários Leves (Bottom 30%):** Clientes novos ou que pedem muito esporadicamente.
* **Racional Estratégico:**
    * A campanha claramente aumentou a frequência geral. O racional desta segmentação é descobrir a **origem desse crescimento**. Queremos responder: **"A campanha foi mais eficaz para criar novos hábitos em clientes pouco engajados ou para acelerar ainda mais os clientes que já eram leais?"**. A resposta nos diz se a campanha é uma ferramenta de **ativação** ou de **fidelização acelerada**, o que impacta diretamente o ROI de cada grupo.

#### **2. Segmentação por Perfil de Gasto (Ticket Médio)**

* **Critérios de Criação:**
    * Calculamos o valor do ticket médio de cada usuário.
    * Utilizamos a **mediana** como ponto de corte para criar dois grupos:
        * **Clientes de Alto Valor:** Usuários que gastam, em média, mais do que a metade da base de clientes.
        * **Clientes de Baixo Valor:** Usuários que gastam, em média, menos do que a metade da base.
* **Racional Estratégico:**
    * Vimos que o ticket médio geral não mudou. O racional aqui é investigar se a campanha teve um apelo diferente para perfis de gasto distintos. A pergunta a ser respondida é: **"O cupom de frequência atraiu mais os 'caçadores de promoção' (focados em economia) ou também engajou clientes que já possuem um ticket médio mais alto?"**. Isso nos ajuda a entender a sensibilidade de cada perfil ao incentivo e o impacto na rentabilidade.

---

### **c) Análise (Hipotética) dos Resultados do Teste por Segmento**

Após rodar o código que está logo abaixo, obtemos os dados que estão nas tabelas para cada segmento. Abaixo, apresento a **análise dos insights**.

#### **Análise por Segmento de FREQUÊNCIA**

| Segmento | Grupo | Pedidos por Usuário (Lift da Campanha) |
| :--- | :--- | :--- |
| **Heavy User** | Controle | 12.5 |
| | Target | 13.1 **(+5%)** |
| **Usuário Casual** | Controle | 5.0 |
| | Target | 5.8 **(+16%)** |
| **Usuário Leve** | Controle | 1.5 |
| | Target | 2.2 **(+47%)** |

* **Insight:** A campanha teve um impacto massivo e decrescente. O maior efeito (+47%) foi sobre os **Usuários Leves**, seguido pelos Casuais. Nos Heavy Users, o efeito foi marginal.
* **Recomendação Estratégica:** A campanha é uma **ferramenta espetacular de ativação e criação de hábito**. O investimento em cupons se justifica muito mais em clientes de baixa frequência, onde o potencial de mudança de comportamento é maior. Para Heavy Users, o custo pode não valer a pena, e outras estratégias de reconhecimento (benefícios não-financeiros, exclusividade) podem ser mais eficazes.

#### **Análise por Segmento de GASTO (Ticket Médio)**

| Segmento | Grupo | Pedidos por Usuário (Lift da Campanha) | Rentabilidade (ROI Hipotético) |
| :--- | :--- | :--- | :--- |
| **Alto Valor** | Controle | 4.5 | Base |
| | Target | 4.9 **(+9%)** | **-40%** |
| **Baixo Valor** | Controle | 4.0 | Base |
| | Target | 4.7 **(+18%)** | **-95%** |

* **Insight:** O incentivo foi mais atraente para o segmento de **Baixo Valor**, que mostrou um lift maior na frequência de pedidos. No entanto, é exatamente nesse grupo que a rentabilidade é pior, pois o valor fixo do cupom representa uma porcentagem muito maior do valor do pedido, tornando o prejuízo por pedido maior.
* **Recomendação Estratégica:** O cupom de valor fixo não é eficaz para engajar o público de "Alto Valor" de forma rentável. Para este segmento, seria mais inteligente testar outros tipos de incentivos, como **"Frete Grátis acima de R\$X"** ou **"15% de desconto em pedidos acima de R\$Y"**, que incentivam o aumento do ticket médio e se alinham melhor ao seu perfil de gasto.

---

### **Conclusão Final:**

A segmentação nos mostra que uma abordagem "one-size-fits-all" é ineficiente. Ao analisar os resultados por perfil, podemos evoluir de uma campanha única para um portfólio de ações personalizadas, otimizando o investimento e impulsionando um crescimento mais inteligente e lucrativo.

## O Desafio – Próximos Passos Ifood (ETAPA 3)


---

### **3. Próximos Passos e Recomendações Estratégicas**


#### **Resumo Executivo | Data Analysis - iFood**

Nossa análise aprofundada da campanha de cupons revelou uma dualidade crítica: enquanto a iniciativa foi um **sucesso em engajar e reter nossa base de clientes já ativa**, ela se mostrou **financeiramente insustentável**. A aplicação de uma estratégia "one-size-fits-all" gerou um prejuízo significativo ao subsidiar um comportamento que não mudou no segmento menos engajado.

Propomos uma mudança de paradigma: abandonar a abordagem única e adotar um **portfólio de estratégias personalizadas por segmento**. As recomendações abaixo visam **otimizar o investimento, focar em ativação real e transformar a alavanca de frequência em um motor de crescimento lucrativo**, com um impacto financeiro positivo estimado em milhões de reais por ciclo de campanha.

---

#### **Diagnóstico Estratégico (O que Aprendemos)**

1.  **A Campanha é um "Acelerador de Leais", não um "Ativador de Novatos":** O incentivo de frequência funcionou muito bem para quem já tinha o hábito de pedir (`Heavy Users` e `Casuais`), mas falhou em converter `Usuários Leves`, onde o lift de comportamento foi próximo de zero.
2.  **O Modelo "One-Size-Fits-All" é Ineficiente e Caro:** Direcionar o mesmo incentivo para toda a base resultou em um desperdício de recursos com o segmento que não reagiu, tornando o ROI geral da campanha massivamente negativo (-89% com a premissa de R$10 por cupom).
3.  **Diferentes Perfis Exigem Diferentes Incentivos:** O apelo do cupom de frequência foi similar entre clientes de alto e baixo ticket médio, mas a rentabilidade é drasticamente diferente. Isso prova que precisamos de um arsenal de incentivos mais sofisticado.

---

### **Plano de Ação: Próximos Passos Recomendados**

Propomos uma abordagem em três frentes, agindo de forma imediata para otimizar custos e de forma estratégica para construir um crescimento sustentável.

#### **Ação 1: Otimização Imediata - Focar o Investimento Onde Funciona**

* **Ação:** **Pausar imediatamente a oferta do cupom de frequência para o segmento de "Usuários Leves"**.
* **Racional:** Nossa análise mostrou que o lift de pedidos neste grupo foi de apenas +1.3%. Continuar a investir neles com esta campanha é financeiramente insustentável.
* **Previsão de Impacto:**
    * **Financeiro (Economia Direta):** Com base nos dados do teste, o segmento de "Usuários Leves" no grupo `target` continha ~215 mil usuários que fizeram ~340 mil pedidos. Com um custo de R$ 10 por cupom, essa ação representa uma **economia imediata de R$ 3,4 milhões por ciclo de campanha**, eliminando um investimento com retorno praticamente nulo.

#### **Ação 2: Nova Estratégia de Ativação (Para "Usuários Leves")**

* **Ação:** Desenhar e testar uma **nova campanha focada exclusivamente na ativação** deste segmento, já que o incentivo de frequência não funcionou.
* **Proposta de Novo Teste A/B:**
    * **Hipótese:** "Para usuários com poucos pedidos, um incentivo de alto valor na *próxima compra* é mais eficaz para quebrar a barreira inicial do que um pequeno incentivo de frequência."
    * **Grupos:**
        * **Grupo A (Controle):** Usuários Leves sem nenhuma ação.
        * **Grupo B (Variante):** Oferecer **"Frete Grátis nos próximos 2 pedidos"**.
        * **Grupo C (Variante):** Oferecer **"25% de desconto no próximo pedido acima de R$ 50"**.
    * **Métrica de Sucesso:** Taxa de conversão para o segundo e terceiro pedido.
* **Previsão de Impacto:**
    * **Estratégico:** Aumentar a taxa de ativação de "Usuários Leves" para "Usuários Casuais" em **15%**. Cada usuário ativado representa um aumento significativo no LTV (Valor do Ciclo de Vida do Cliente), transformando um segmento de baixo valor em uma base de clientes recorrentes.

#### **Ação 3: Estratégia de Rentabilização (Para "Heavy Users" e "Casuais")**

* **Ação:** Otimizar a campanha para os segmentos que responderam bem, com o objetivo de **torná-la lucrativa**.
* **Proposta de Novo Teste A/B:**
    * **Hipótese:** "É possível manter um lift de frequência positivo nestes segmentos com um custo de incentivo menor, alcançando um ROI positivo."
    * **Grupos:**
        * **Grupo A (Controle):** Sem cupom.
        * **Grupo B (Variante):** Cupom de **R$ 4,00** para o próximo pedido.
        * **Grupo C (Variante):** Sistema de gamificação com acúmulo de pontos que resultem em um benefício equivalente a ~R$ 3,00 por pedido.
    * **Métrica de Sucesso:** **Margem de Contribuição por Usuário** e **ROI da Campanha**.
* **Previsão de Impacto:**
    * **Financeiro:** Transformar uma iniciativa com ROI de -89% em uma com **ROI positivo (meta: +20%)**. Isso não só manteria o engajamento desses clientes valiosos, mas transformaria a campanha de um centro de custo em um **motor de crescimento lucrativo**.

---

### **Melhorias no Processo de Testes A/B**

Para garantir que não repitamos os mesmos erros, proponho duas melhorias em nosso processo de experimentação:

1.  **Segmentação como Padrão:** Toda análise de teste A/B deve, por padrão, incluir uma análise segmentada pós-teste para entender os efeitos em diferentes perfis de usuário.
2.  **Métricas Financeiras como Critério:** Todo teste deve ter, além de uma métrica de produto primária (ex: frequência), uma **métrica de "guarda" financeira** (ex: Margem de Contribuição ou ROI). O sucesso de um teste só será declarado se a métrica de produto subir **sem prejudicar** a métrica financeira.

Ao adotar estes próximos passos, o iFood evoluirá de uma estratégia de crescimento baseada em volume para um modelo sofisticado, segmentado e focado em **crescimento rentável e sustentável**.

## Gerando o relatório executivo em PDF

In [ ]:
# ==============================================================================
# 0. INSTALAÇÃO DAS DEPENDÊNCIAS
# ==============================================================================
# FPDF2 para criar o PDF, Matplotlib para os gráficos e Pandas para a transição.
!pip install -q fpdf2 matplotlib pandas


# ==============================================================================
# 1. SETUP E ANÁLISE COM PYSPARK
# ==============================================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, when, percent_rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("GeneratePDFReport").getOrCreate()

# --- Agregação por Usuário e Segmentação ---
df_user_features = df_ab_orders.groupBy("customer_id", "is_target").agg(
    count("order_id").alias("total_pedidos"),
    avg("order_total_amount").alias("ticket_medio_usuario")
)

window_freq = Window.partitionBy("is_target").orderBy("total_pedidos")
df_user_features = df_user_features.withColumn("rank_freq", percent_rank().over(window_freq))
df_user_features = df_user_features.withColumn("segmento_frequencia",
    when(col("rank_freq") >= 0.8, "1. Heavy User")
    .when((col("rank_freq") >= 0.3) & (col("rank_freq") < 0.8), "2. Casual")
    .otherwise("3. Leve")
)

# --- Análise Final por Segmento ---
df_analise_frequencia = df_user_features.groupBy("is_target", "segmento_frequencia").agg(
    count("customer_id").alias("qtd_usuarios"),
    avg("total_pedidos").alias("pedidos_por_usuario")
).orderBy("segmento_frequencia", "is_target")

# Coletando para o Pandas
frequencia_pd = df_analise_frequencia.toPandas()

# --- Cálculos Financeiros (usando os números da nossa análise) ---
receita_incremental = 2325811
custo_campanha = 10683725 # Assumindo cupom de R$ 5
resultado_liquido = receita_incremental - custo_campanha


# ==============================================================================
# 2. GERAÇÃO DOS GRÁFICOS COM MATPLOTLIB
# ==============================================================================
import matplotlib.pyplot as plt
import numpy as np

# Gráfico 1: Lift de Frequência por Segmento
def create_segment_lift_chart(df_pd):
    control_data = df_pd[df_pd['is_target'] == 'control'].set_index('segmento_frequencia')['pedidos_por_usuario']
    target_data = df_pd[df_pd['is_target'] == 'target'].set_index('segmento_frequencia')['pedidos_por_usuario']
    lift = ((target_data / control_data) - 1) * 100
    lift = lift.sort_index()

    plt.figure(figsize=(10, 6))
    bars = plt.bar(lift.index, lift.values, color=['#EA1D2C', '#F37A20', '#F3C120'])
    plt.ylabel('Aumento Percentual (%) nos Pedidos por Usuário', fontsize=12)
    plt.title('Impacto da Campanha por Segmento de Frequência', fontsize=16, weight='bold')
    plt.axhline(0, color='grey', linewidth=0.8)
    plt.xticks(fontsize=12)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + 1, f'{yval:.1f}%', ha='center', va='bottom', fontsize=12, weight='bold')
    plt.tight_layout()
    plt.savefig('grafico_segmentos.png')
    plt.close()

# Gráfico 2: Análise Financeira
def create_financial_chart(receita, custo, resultado):
    data = {'Receita Incremental': receita, 'Custo da Campanha': -custo, 'Resultado Líquido': resultado}
    colors = ['#4CAF50', '#F44336', '#2196F3'] if resultado > 0 else ['#4CAF50', '#F44336', '#F44336']

    plt.figure(figsize=(10, 6))
    bars = plt.bar(data.keys(), data.values(), color=colors)
    plt.ylabel('Valor (R$)', fontsize=12)
    plt.title('Análise Financeira da Iniciativa', fontsize=16, weight='bold')
    plt.axhline(0, color='grey', linewidth=0.8)
    plt.xticks(fontsize=12)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval + (5e5 * np.sign(yval)), f'R$ {yval/1e6:.2f}M', ha='center', va='center', fontsize=12, weight='bold', color='white')
    plt.tight_layout()
    plt.savefig('grafico_financeiro.png')
    plt.close()

# Gerar os gráficos
create_segment_lift_chart(frequencia_pd)
create_financial_chart(receita_incremental, custo_campanha, resultado_liquido)


# ==============================================================================
# 3. MONTAGEM DO RELATÓRIO EM PDF COM FPDF2
# ==============================================================================
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Relatório Estratégico iFood - Análise de Teste A/B', 0, 1, 'C')
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 14)
        self.set_fill_color(234, 29, 44) # Vermelho iFood
        self.set_text_color(255, 255, 255)
        self.cell(0, 10, title, 0, 1, 'L', fill=True)
        self.ln(5)
        self.set_text_color(0, 0, 0)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 7, body)
        self.ln()

    def add_insight_bullet(self, text):
        self.set_font('Arial', 'B', 12)
        self.cell(10, 7, '•', 0, 0, 'L')
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 7, text)


# --- Conteúdo do Relatório ---
pdf = PDF()
pdf.add_page()

# Resumo Executivo
pdf.chapter_title('1. Resumo Executivo e Principal Recomendação')
pdf.chapter_body(
    "A análise da campanha de cupons revela que, embora tenha sido um sucesso em engajar nossa base de clientes já ativa, ela se mostrou financeiramente inviável e ineficaz para ativar usuários de baixa frequência. O custo do incentivo superou a receita incremental gerada."
)
pdf.add_insight_bullet(
    "Recomendação Principal: Propomos uma mudança estratégica, abandonando a abordagem 'one-size-fits-all' para um portfólio de ações segmentadas, começando por pausar a campanha para 'Usuários Leves' e otimizando o custo para os demais, visando transformar a iniciativa em um motor de crescimento lucrativo."
)

# Diagnóstico
pdf.chapter_title('2. Diagnóstico: O Que Aprendemos com o Teste?')
pdf.chapter_body(
    "A análise segmentada nos permitiu identificar exatamente onde a campanha funcionou e onde falhou. O gráfico abaixo ilustra a descoberta chave:"
)
pdf.image('grafico_segmentos.png', x=None, y=None, w=180)
pdf.add_insight_bullet(
    "Acelerador de Leais: A campanha teve um impacto forte e positivo nos 'Heavy Users' e 'Casuais', que já tinham o hábito de pedir. O incentivo funcionou como um acelerador para este grupo."
)
pdf.add_insight_bullet(
    "Falha na Ativação: O efeito foi praticamente nulo nos 'Usuários Leves'. A campanha não conseguiu criar novos hábitos, indicando que este público precisa de um tipo diferente de estímulo."
)

# Análise Financeira
pdf.chapter_title('3. Análise Financeira: O Retorno sobre o Investimento')
pdf.chapter_body(
    "Apesar do sucesso em engajamento, o modelo financeiro se mostrou insustentável. O gráfico abaixo resume o resultado líquido da campanha, com base na premissa de R$ 5 de custo por cupom."
)
pdf.image('grafico_financeiro.png', x=None, y=None, w=180)
pdf.add_insight_bullet(
    f"Resultado Negativo: A campanha gerou uma receita incremental de R$ {receita_incremental/1e6:.2f}M, mas teve um custo de R$ {custo_campanha/1e6:.2f}M, resultando em um prejuízo de R$ {abs(resultado_liquido/1e6):.2f}M."
)
pdf.add_insight_bullet(
    "Diagnóstico: O custo fixo do cupom por pedido é muito alto em relação à receita gerada, especialmente nos segmentos de menor ticket médio."
)


# Próximos Passos
pdf.add_page()
pdf.chapter_title('4. Plano de Ação Estratégico: Nossos Próximos Passos')
pdf.chapter_body(
    "Com base nestes insights, recomendamos um plano de ação em três frentes para transformar esta iniciativa em uma alavanca de crescimento rentável e inteligente:"
)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, "Ação 1: Otimização Imediata (Foco: Usuários Leves)", 0, 1, 'L')
pdf.chapter_body(
    "Pausar a oferta de cupons de frequência para o segmento de 'Usuários Leves'. Em vez disso, testar campanhas de ativação, como 'Frete Grátis nos 2 primeiros pedidos' ou '25% de desconto na primeira compra', focadas em garantir uma excelente primeira experiência."
)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, "Ação 2: Rentabilização (Foco: Heavy Users e Casuais)", 0, 1, 'L')
pdf.chapter_body(
    "Para os grupos que responderam bem, o desafio é a lucratividade. Propomos um novo teste A/B com incentivos de custo menor (ex: cupom de R$ 3) ou condicionados a um ticket médio maior (ex: '10% de desconto em pedidos acima de R$ 80')."
)
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, "Ação 3: Melhoria de Processo", 0, 1, 'L')
pdf.chapter_body(
    "Institucionalizar a análise segmentada como um padrão para todos os testes A/B futuros e sempre incluir uma métrica de 'guarda' financeira (ex: Margem de Contribuição) como critério de sucesso."
)

# Salvar o PDF

# Mensagem de ajuda com exemplos claros para o usuário
prompt_de_caminho = """
======================================================================
O relatório em PDF está pronto para ser salvo.
Por favor, digite o caminho completo onde o arquivo será salvo,
incluindo o nome do arquivo (ex: Relatorio_iFood.pdf).
======================================================================

DICAS PARA PREENCHER O CAMINHO:

-> Para Windows:
   Use barras normais / (recomendado) ou barras duplas \\\\.
   Exemplo 1: C:/Users/SeuNome/Documentos/Relatorio_Final.pdf
   Exemplo 2: C:\\Users\\SeuNome\\Desktop\\Analise_iFood.pdf

-> Para macOS / Linux:
   Use a barra normal / e o atalho ~ para sua pasta de usuário.
   Exemplo 1: /Users/SeuNome/Desktop/Relatorio_Final.pdf
   Exemplo 2: ~/Documentos/Analise_iFood.pdf

Digite o caminho desejado e pressione Enter: """

# Solicitar o caminho completo ao usuário
caminho_salvar = input(prompt_de_caminho)

try:
    # Salvar o PDF no caminho especificado pelo usuário
    pdf.output(caminho_salvar)
    print(f"\n✅ Sucesso! O relatório foi salvo em: {caminho_salvar}")
except Exception as e:
    print(f"\n❌ Erro ao salvar o arquivo! Verifique se o caminho está correto e se a pasta existe.")
    print(f"Detalhe do erro: {e}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 10.6 MB/s eta 0:00:00
